# mypulp

## Linear Optimization 線形最適化

In [1]:
from mypulp import *

In [2]:
model = Model('lo1')
model

lo1:
MINIMIZE
None
VARIABLES

In [3]:
x1 = model.addVar(vtype='C', name='x1')
x1

Variable: x1 lb=0.0 ub=1e+100 vtype=C X=None RC=None

In [4]:
x2 = model.addVar(name='x2')
x3 = model.addVar(name='x3')

In [5]:
x1, x2, x3

(Variable: x1 lb=0.0 ub=1e+100 vtype=C X=None RC=None,
 Variable: x2 lb=0.0 ub=1e+100 vtype=C X=None RC=None,
 Variable: x3 lb=0.0 ub=1e+100 vtype=C X=None RC=None)

In [6]:
model

lo1:
MINIMIZE
None
VARIABLES

In [7]:
model.update()

In [8]:
model.addConstr(2*x1+x2+x3 <= 60)

2*x1 + 1*x2 + 1*x3 + -60 <= 0

In [9]:
model

lo1:
MINIMIZE
None
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 1e+100 Continuous

In [10]:
model.addConstr(x1+2*x2+x3 <= 60)
model.addConstr(x3<=30)
model

lo1:
MINIMIZE
None
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

c_2: x1 + 2 x2 + x3 <= 60

c_3: x3 <= 30

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 1e+100 Continuous

In [11]:
model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)
model

lo1:
MAXIMIZE
15*x1 + 18*x2 + 30*x3 + 0
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

c_2: x1 + 2 x2 + x3 <= 60

c_3: x3 <= 30

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 1e+100 Continuous

In [12]:
%%time
model.optimize()

CPU times: user 3.31 ms, sys: 8.06 ms, total: 11.4 ms
Wall time: 27 ms


In [13]:
model.ObjVal

1230.0

In [14]:
model.getVars()

[Variable: x1 lb=0.0 ub=1e+100 vtype=C X=10.0 RC=0.0,
 Variable: x2 lb=0.0 ub=1e+100 vtype=C X=10.0 RC=0.0,
 Variable: x3 lb=0.0 ub=1e+100 vtype=C X=30.0 RC=0.0]

In [15]:
for v in model.getVars():
    print(v.VarName, v.X)

x1 10.0
x2 10.0
x3 30.0


In [16]:
model.getConstrs()

[2*x1 + 1*x2 + 1*x3 + -60 <= 0, 1*x1 + 2*x2 + 1*x3 + -60 <= 0, 1*x3 + -30 <= 0]

In [17]:
for c in model.getConstrs():
    print(c.ConstrName, c.Pi)

c_1 4.0
c_2 7.0
c_3 19.0


## Integer Optimization 整数最適化

In [18]:
multidict

<function mypulp.multidict(dic)>

In [19]:
name, height, weight = multidict({'Taro':[1,2], 'Jiro':[3,4], 'Sabu':[5,6]})
name, height, weight

(['Taro', 'Jiro', 'Sabu'],
 {'Taro': 1, 'Jiro': 3, 'Sabu': 5},
 {'Taro': 2, 'Jiro': 4, 'Sabu': 6})

In [20]:
def example():
    J, v = multidict({1:16, 2:19, 3:23, 4:28})
    a = {(1,1):2, (1,2):3, (1,3):4, (1,4):5,
        (2,1):3000, (2,2):3500, (2,3):5100, (2,4):7200}
    I, b = multidict({1:7, 2:10000})
    return I, J, v, a, b

In [22]:
example()

([1, 2],
 [1, 2, 3, 4],
 {1: 16, 2: 19, 3: 23, 4: 28},
 {(1, 1): 2,
  (1, 2): 3,
  (1, 3): 4,
  (1, 4): 5,
  (2, 1): 3000,
  (2, 2): 3500,
  (2, 3): 5100,
  (2, 4): 7200},
 {1: 7, 2: 10000})

In [23]:
def mkp(I, J, v, a, b):
    model = Model('mkp')
    x = {}
    for j in J:
        x[j] = model.addVar(vtype='B', name='x(%d)'%j)
    model.update()
    for i in I:
        model.addConstr(quicksum(a[i,j]*x[j] for j in J)<=b[i])
    model.setObjective(quicksum(v[j]*x[j] for j in J), GRB.MAXIMIZE)
    model.update()
    return model

In [24]:
quicksum([2*x1, x2, x3])

2*x1 + 1*x2 + 1*x3 + 0

In [26]:
I, J, v, a, b = example()
model = mkp(I, J, v, a, b)
model.update()
model.write('mkp.lp')

In [27]:
model.update()
model.write('mkp.mps')

In [29]:
%%time
model.optimize()

CPU times: user 4.56 ms, sys: 8.27 ms, total: 12.8 ms
Wall time: 39.2 ms


In [30]:
model.ObjVal

42.0

In [33]:
for v in model.getVars():
    print(v.VarName, v.X)


x(1) 0.0
x(2) 1.0
x(3) 1.0
x(4) 0.0


## hoge